In [132]:
# %%writefile alpha_base10.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
    
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import preprocessing
from sklearn import linear_model
import tools.Pretreat_Tools as pretreat
import tools.Sample_Tools as smpl

from base.JuUnits import excute_for_multidates



In [135]:
from tools.Cacher import (CACHE_TYPE, load_cache,load_caches_adv)
import tools.Pretreat_Tools as pretreat
import QUANTAXIS as QA


%load_ext autoreload
%autoreload 2
%aimport tools.Cacher,tools.Sample_Tools

stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(stock_df)
# stock_df = pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),['all_train_qfq','all_tail_qfq','all_older_qfq']))).sort_index()
ret_fs = pd.read_csv(module_path+'/data/static/china10yearbond.csv').set_index('date').sort_index()
ret_fs = (ret_fs['high'].astype(np.float32)+ret_fs['low'].astype(np.float32))/2 * 0.01
ret_fs_daily = ret_fs/252
# ## 日无风险回报
ret_fs_daily


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:

ret_t = smpl.get_current_return(stock_df,'close')

# 超额回报
ret_t_excess = ret_t.groupby(pd.Grouper(level='date', freq='1M')).apply(
        lambda x:(x-ret_fs_daily.get(x.index[0][0].strftime('%Y-%m'),default=ret_fs_daily[-1])))

# 市值
market_value_t = stock_df['totalCapital']*stock_df['close']
market_value_t.name='market_value'

# 计算市场（平均）收益
def cal_ret_market(market_value,ret_excess_data):
    ##  不取对数有时候有精度问题,权重不是精确1
    market_value_log = np.log(market_value)
    weight = market_value_log / market_value_log.sum()
    ret_market_f = (ret_excess_data * weight).sum()
    return ret_market_f

# 市场收益，全市场收益加权平均
ret_excess_market_t = excute_for_multidates(ret_t_excess,
                                     lambda ret: cal_ret_market(market_value_t.loc[ret.index[0][0]],ret), 
                                     level=0)


def camp_beta_alpha(ret_excess,ret_excess_market):
    # 5年daily单核执行约35分钟
    window=252
    half_life_window = 63
    half_life_ = list(map(lambda n:0.5**(n/half_life_window),range(1,window+1)))[::-1]
    half_life_weight = half_life_/np.sum(half_life_)

    model = linear_model.LinearRegression(fit_intercept=True)
    res_tmp = []
    def reg(ret_t_ex):
        # print()
        # assert False,None
        X = ret_excess_market[ret_t_ex.index.get_level_values(0)]
        res = model.fit(X.values.reshape(-1, 1),
                        ret_t_ex.values.reshape(-1, 1), 
                        sample_weight=half_life_weight)
        
        predict = model.predict([[X[-1]]])
        ret_excess_residual = ret_t_ex[-1] - float(predict)
        
        res_tmp.append({'date':ret_t_ex.index[-1][0],
                        'code':ret_t_ex.index[-1][1], 
                        'beta':float(res.coef_), 
                        'alpha':float(res.intercept_),
                        'ret_excess_residual':ret_excess_residual
                       })
        return 0
    
    ret_excess.dropna().groupby(level=1,group_keys=False).apply(
            lambda x:x.rolling(window).apply(reg))

    res_final = pd.DataFrame(res_tmp)
    res_final.set_index(['date', 'code'], inplace=True)
    res_final = res_final.sort_index()
    return res_final

# camp_beta_alpha(ret_t_excess.loc[(slice(None),['000001','000002'])],ret_market_t)
def generat_tmpxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx():
    pd.DataFrame(res_final['beta']).reset_index().to_feather('./beta_camp.feather')
    pd.DataFrame(res_final['alpha']).reset_index().to_feather('./alpha_camp.feather')

def momentum(ret,ret_fs):
    ret_excess = ret.groupby(pd.Grouper(level='date', freq='1M')).apply(
            lambda x:np.log(1+x)-np.log(1+ret_fs.get(x.index[0][0].strftime('%Y-%m'),default=ret_fs[-1])))

    def calc_(data,window=252,half_life_window=126):
        if len(data) < 253:
            return None
        ewma = data.rolling(window).apply(
                        lambda xx:(xx.ewm(adjust=False,halflife=126).mean()[-1]))
        return ewma.rolling(11).mean().shift(11)

    mom = excute_for_multidates(ret_excess.dropna(), lambda x:calc_(x), level='code')
          
    return mom

def size(stock_data):
    mv = np.log(stock_data['totalCapital']*stock_data['close'])
    return mv

def bp(stock_data):
    data = smpl.add_report_inds(stock_data[['close']],'netAssetsPerShare')
    bp = data['close']/data['netAssetsPerShare']
    return bp


In [78]:
def earnings_yield(ret,market_value,stock_industry):
    # Earnings Yield 收益因子
    # EARNYILD = 0.68*EPIBS + 0.11*ETOP + 0.21*CETOP
    # EPIBS ：分析师预测的 EP （ earnings to price ）。
    # ETOP ： ttm-ep ，最近 12 个月的总盈利除以当前总市值。
    # CETOP ：最近 12 个月的运营现金流处于当前总市值。
    
    mv = market_value#_t.loc[(slice(None),['000001','000008','600600','000729'])]
    codes = mv.index.get_level_values(1).unique().tolist()
    date_ = mv.index.get_level_values(0)
    date_start = str(int(date_.min().strftime("%Y"))-1)
    date_end = date_.max().strftime("%Y")

    # # 利润总额  经营活动产生的现金流量净额  
    report_df = QA.QA_fetch_financial_report_adv(codes, date_start, date_end,ltype='EN').data[['totalProfit','netCashFlowsFromOperatingActivities']]


    # 年报转累进转当季
    report_df = excute_for_multidates(report_df,
                                      lambda stock:stock.groupby(pd.Grouper(level='report_date', freq='1Y')).apply(
                                      lambda x:x.diff(1).fillna(x)),level='code')

    # 四季（年）滚动加总，“最近12个月”
    report_df = excute_for_multidates(report_df,lambda x:x.rolling(4).sum(),level='code')

    data_ = excute_for_multidates(pd.concat([mv,report_df], axis=1),lambda x:x.fillna(method='ffill'),level='code').loc[mv.index].sort_index()
    
    ETOP = data_['totalProfit']/data_['market_value']
    CETOP = data_['netCashFlowsFromOperatingActivities']/data_['market_value']


    # # EPIBS 分析师的期望暂时用季度收益斜率+行业季度收益斜率来代表。
    m = linear_model.LinearRegression(fit_intercept=True)
    def ret_cum_reg(ret,window=63):
            # print()
            # assert False,None
            def reg(window_slice):
                res = m.fit(np.arange(0.01,0.01*window+0.01,0.01).reshape(-1, 1), 
                            window_slice.values.reshape(-1, 1)
                           )
                # print(window_slice.index[-1],float(res.coef_),float(res.intercept_))
                return float(res.coef_)

            k = np.log(1+ret).rolling(window).apply(lambda x:reg(x))
            return k

    # ret_expect = excute_for_multidates(ret.loc[(slice(None),['000001','000008','600600','000729'])], lambda x:ret_cum_reg(x),level='code').sort_index()
    ret_expect = excute_for_multidates(ret, lambda x:ret_cum_reg(x),level='code').sort_index()

    # ret_industry = pd.concat([ret,stock_industry], axis=1).loc[(slice(None),['000001','000008','600600','000729']),:].sort_index()
    ret_industry = pd.concat([ret,stock_industry], axis=1).sort_index()
    ret_industry_meam = ret_industry.reset_index().set_index(['date','industry']).groupby(level=[0,1]).mean()
    ret_industry_expect = excute_for_multidates(ret_industry_meam, lambda x:ret_cum_reg(x),level='industry')

    EPIBS = ret_expect + ret_industry_expect.loc[list(zip(ret_industry.index.get_level_values(0),ret_industry['industry']))]['ret'].values

    # # # 测试
    # with pd.option_context("display.max_rows", None, "display.max_columns", None):
    #     x = pd.DataFrame(ret_industry_expect.loc[list(zip(ret_industry.index.get_level_values(0),ret_industry['industry']))].values,index=ret_industry.index)
    #     display(pd.concat([ret_industry,ret_expect,x],axis=1))

    EARNYILD = 0.68*EPIBS + 0.11*ETOP + 0.21*CETOP
    
    return EARNYILD

# x = earnings_yield(ret_t.loc[(slice(None),['000001','000002'])],market_value_t.loc[(slice(None),['000001','000002'])],stock_df['industry'].loc[(slice(None),['000001','000002'])])

In [70]:
# def liquidity():

# Liquidity 因子是三个子因子的简单加权：
# LIQUIDTY = 0.35*STOM + 0.35*STOQ + 0.30*STOA 
# STOM: 月换手率：ST(1)
# STOQ ：三个月的平均月换手率：ST(3)
# STOA ：十二个月的平均月换手率：ST(12)


In [172]:
# test = camp_beta_alpha(ret_t_excess.loc[(slice(None),['000001','000002'])],ret_excess_market_t)
# test
# tttttttttt = stock_df.loc[(slice(None),['000001','000008','600600','000729']),:]

In [194]:
# RESVOL 波动因子
# RESVOL = 0.74*DASTD + 0.16 *CMRA + 0.10*HSIGMA
# DASTD ： 过去 252 个交易日的超额盈利的加权波动率，加权系数半衰期为 42 天。
# CMRA ： 过去 12 个月的最高收益率和最低收益率的差，相当于按月算的波动幅度。
# HSIGMA ： 在计算 beta 时残差项的标准差，使用 252 个交易日的样本，半衰期为 63 天。
# 最后RESVOL对beta做正交化

window=252
half_life_window = 42
half_life_ = list(map(lambda n:0.5**(n/half_life_window),range(1,window+1)))[::-1]
half_life_weight = half_life_/np.sum(half_life_)
    
DASTD = excute_for_multidates(ret_t_excess.loc[(slice(None),['000001','000002'])], 
                              lambda stock:stock.rolling(window).apply(
                                  lambda x:((x-x.mean())**2).dot(half_life_weight)**(0.5)
                              ) ,level='code')

date        code  
2017-01-03  000001         NaN
            000002         NaN
2017-01-04  000001         NaN
            000002         NaN
2017-01-05  000001         NaN
                        ...   
2021-12-29  000002    0.024942
2021-12-30  000001    0.020634
            000002    0.024788
2021-12-31  000001    0.020625
            000002    0.024981
Name: ret, Length: 2429, dtype: float64

In [69]:

sys.getsizeof(int(1)),sys.getsizeof(float(1)),sys.getsizeof(np.float64(1)),sys.getsizeof(np.float32(1)),sys.getsizeof(np.float16(1)),sys.getsizeof(np.int32(1)),sys.getsizeof(np.int64(1))

(28, 24, 32, 28, 26, 28, 32)

In [188]:
# sys.getsizeof(ret_fs_daily)/1024/1024
# 732.7092390060425
# (test['beta']**2)**(0.5)
np.square(test['beta'])

date        code  
2018-01-12  000001    0.094098
2018-01-15  000001    0.026095
2018-01-16  000001    0.024547
2018-01-17  000001    0.024459
2018-01-18  000001    0.028328
                        ...   
2021-12-29  000002    0.259874
2021-12-30  000001    0.295442
            000002    0.243542
2021-12-31  000001    0.277089
            000002    0.267609
Name: beta, Length: 1925, dtype: float64

In [136]:
stock_df.info()
# memory usage: 438.9+ MB



<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3247889 entries, (Timestamp('2017-01-03 00:00:00'), '000001') to (Timestamp('2021-12-31 00:00:00'), '605599')
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   open                    float32
 1   high                    float32
 2   low                     float32
 3   close                   float32
 4   volume                  int32  
 5   amount                  float64
 6   adj                     float32
 7   shares                  float64
 8   lshares                 float64
 9   totalCapital            float64
 10  liquidity_totalCapital  float64
 11  industry                object 
dtypes: float32(5), float64(5), int32(1), object(1)
memory usage: 235.5+ MB
